# Recommendations

## 개요와 배경

### 개요

추천 시스템의 중요성이 부각되는 시대: 포탈 사이트, 전자 상거래 업체, 영화/음악 콘텐츠 제공 업체 등 적극적으로 활용

의의: 사용자 자신도 몰랐던 취향의 발견, 그에 맞는 콘텐츠 추천 (나아가 서비스의 지속)

### 온라인 스토어

많은 카테고리와 상품 속에서 제품의 추천은 매우 중요 ⇒ 추천 엔진은 사용자의 잠재적 수요 욕구를 찾아내어 추천

사용자 구매 상품, 사용자 조회 상품, 사용자 장바구니, 사용자의 평점, 사용자 스스로 작성한 취향, 사용자의 클릭

### 추천 시스템의 유형

콘텐츠 기반 필터링

협업 필터링

- 최근접 이웃 협업 필터링
- 잠재 요인 협업 필터링 (대부분의 온라인 스토어에서 사용 중)

## 콘텐츠 기반 필터링 추천 시스템

특정한 아이템을 선호하는 경우, 비슷한 콘텐츠를 가진 다른 아이템을 추천

사용자가 좋은 평가를 내린 제품의 속성들을 기반으로 속성 중 유사한 것들을 추천

## 최근접 이웃 협업 필터링

사용자가 아이템에 매긴 평점 정보나 상품 구매 이력과 같은 사용자 행동 양식만을 기반으로 추천 수행

사용자가 평가한 다른 아이템 기반, 다른 평가하지 않은 아이템의 예측 결과 도출

행은 사용자 아이디, 열은 아이템 아이디로 각 사용자의 평점을 나타내는 표가 나와야 가능함

- 사용자 기반: 사용자와 유사한 고객이 구매한 상품을 추천
    - 사용자가 내린 평가를 보았을 때, 가장 유사한 유저를 선정하여 그 유저로부터 평점이 높았던 것을 추천
- 아이템 기반: 이 아이템을 구매한 고객이 구매한 다른 상품을 추천
    - (행과 열이 반대) 아이템의 평점 분포가 가장 유사한 것을 선정하여, 평점이 높을 것으로 보이지만 아직 구매 혹은 관람하지 않은 것을 추천

일반적으로는 사용자 기반보다 아이템 기반 협업 필터링이 정확도가 더 높음

코사인 유사도 이용

## 잠재 요인 협업 필터링

사용자-아이템 평점 매트릭스 속에 숨어 있는 잠재 요인을 추출 - 추천 예측을 할 수 있게 하는 기법

행렬 분해(Matrix Factorization): 행렬을 SVD (차원 감소 기법)로 분해하는 과정에서 잠재 요인 추출

사용자-아이템 행렬 데이터 = 사용자-잠재 요인 행렬 * 잠재 요인-아이템 행렬

분해된 데이터 세트를 다시 내적 곱으로 결합: 사용자가 예측하지 않은 아이템에 대한 평점 도출 방식

### 행렬 분해 이해

SVD, NMF 기법 등이 있음

고차원의 희소 행렬인 R 행렬 = 저차원의 밀집 행렬 P 행렬 * Q 행렬

행렬의 내적으로 비어있는 부분(평가가 이루어지지 않은 부분)에 대한 예측이 가능함

행렬 분해는 확률적 경사 하강법이나 ALS 방식을 이용하여 수행

### 확률적 경사 하강법

1. P와 Q를 임의의 값을 가진 행렬로 설정
2. P * Q.T를 곱해 R을 계산, 오류 값 계산
3. 오류 값 최소화를 위한 적절한 값으로 업데이트
4. 2, 3 반복을 통해 근사화

오류 최소화와 L2 규제를 고려한 비용 함수식

$$
min\sum(r_{(u,i)}-p_uq_i^t)^2+\lambda(||q_i||^2+||p_u||^2)
$$

비용 함수 최적화를 위해 업데이트되는 값

$$
p_u^`=p_u+ƞ(e_{(u,i)}*q_i-\lambda*p_u)
$$

$$
q_i^`=q_i+ƞ(e_{(u,i)}*p_u-\lambda*q_i)
$$

실제 행렬 값과 예측 행렬 값의 차이를 최소화하는 방향성을 가지고 반복적으로 수행

SGD 행렬 분해 구현

In [1]:
import numpy as np

R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN, 3, 4, 4],
              [5, 2, 1, 2, np.NaN]])
num_users, num_items = R.shape
K = 3

np.random.seed(1)
P = np.random.normal(scale=1./K, size=(num_users, K))
Q = np.random.normal(scale=1./K, size=(num_items, K))

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    full_pred_matrix = np.dot(P, Q.T)

    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]

    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [3]:
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0 ]

steps=1000
learning_rate=0.01
r_lambda=0.01

for step in range(steps):
    for i, j, r in non_zeros:
        eij = r - np.dot(P[i, :], Q[j, :].T)
        P[i, :] = P[i, :] + learning_rate * (eij * Q[j, :] - r_lambda * P[i, :])
        Q[j, :] = Q[j, :] + learning_rate * (eij * P[i, :] - r_lambda *Q [j, :])

    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 50) == 0 :
        print("## iteration step:", step, "rmse:", rmse)

## iteration step: 0 rmse: 3.2388050277987723
## iteration step: 50 rmse: 0.4876723101369647
## iteration step: 100 rmse: 0.15643403848192458
## iteration step: 150 rmse: 0.07455141311978064
## iteration step: 200 rmse: 0.043252267985793146
## iteration step: 250 rmse: 0.029248328780879226
## iteration step: 300 rmse: 0.022621116143829507
## iteration step: 350 rmse: 0.019493636196525232
## iteration step: 400 rmse: 0.018022719092132773
## iteration step: 450 rmse: 0.01731968595344283
## iteration step: 500 rmse: 0.016973657887570985
## iteration step: 550 rmse: 0.01679680459589558
## iteration step: 600 rmse: 0.016701322901884634
## iteration step: 650 rmse: 0.016644736912476574
## iteration step: 700 rmse: 0.016605910068210012
## iteration step: 750 rmse: 0.01657420047570488
## iteration step: 800 rmse: 0.01654431582921612
## iteration step: 850 rmse: 0.016513751774735037
## iteration step: 900 rmse: 0.01648146573819507
## iteration step: 950 rmse: 0.01644717168347911


In [4]:
prd_matrix = np.dot(P, Q.T)
print("expected matrix:\n", np.round(prd_matrix, 3))

expected matrix:
 [[3.991 0.897 1.306 2.002 1.663]
 [6.696 4.978 0.979 2.981 1.003]
 [6.677 0.391 2.987 3.977 3.986]
 [4.968 2.005 1.006 2.017 1.14 ]]
